In [1]:
import pandas as pd
from os import listdir
from datetime import datetime, timedelta

In [16]:
def ls(pwd): 
    return listdir(pwd)

def parse_time(x):
    datetime_object = datetime.strptime(x,'%H:%M:%S')
    return datetime_object

def get_index(df):
    news_rows = []
    for row in df.itertuples(index=True):
        index = row.Index
        date_1 = parse_time(df['Timestamp'][index])
        if index < (len(df)-1):
            date_2 =  parse_time( df['Timestamp'][index+1])
        else:
            date_2 =  parse_time( df['Timestamp'][index])
        minutes = ((date_2 - date_1).seconds)/60
        if minutes > 1:
            news_rows.append({'index': index, 'num': minutes-1})
    return news_rows

def get_index_optimizado(df):
    news_rows = []
    for row in df.itertuples(index=True):
        index = row.Index
        date_1 = parse_time(df['Timestamp'][index])
        if index < (len(df)-1):
            date_2 =  parse_time( df['Timestamp'][index+1])
        else:
            date_2 =  parse_time( df['Timestamp'][index])
        minutes = ((date_2 - date_1).seconds)/60
        if (minutes > 1)&(minutes< 1049):
            news_rows.append({'index': index, 'num': minutes-1})
    return news_rows


def create_df(df, news_rows):
    indexs = []
    num_rows = []
    for key in news_rows:
        if key['num'] < 1049:
            indexs.append(key['index'])
            num_rows.append(key['num'])
        else:
            continue

    nf = pd.DataFrame(columns=['Date', 'Timestamp', 'Ticker', 'OpenPrice',
                               'HighPrice', 'LowPrice', 'ClosePrice', 'TotalVolume', 
                               'TotalQuantity', 'TotalTradeCount'])
    index = 0
    for row in df.itertuples(index=True):
        lista= [row.Date, row.Timestamp, row.Ticker,
                row.OpenPrice, row.HighPrice, row.LowPrice, row.ClosePrice,
                row.TotalVolume, row.TotalQuantity, row.TotalTradeCount]
    
        nf.loc[index] = lista
    
        if row.Index in indexs:
            index_array = indexs.index(row.Index)
            index = index + 1
            key = 0
            while key < num_rows[index_array]:
                nf.loc[index] = [row.Date, (parse_time(row.Timestamp) + timedelta(minutes=key+1)).strftime('%H:%M:%S'),
                                 row.Ticker,row.OpenPrice, row.HighPrice, row.LowPrice, row.ClosePrice, 0, 0, 0]
                key = key + 1
                index = index + 1
            
        index = index + 1
    return nf

def optimizacion(df, news_rows):
    indexs = []
    num_rows = []
    for key in news_rows: 
        indexs.append(key['index'])
        num_rows.append(key['num'])
        
    df_final = pd.DataFrame(columns=['Date', 'Timestamp', 'Ticker', 'OpenPrice',
                                     'HighPrice', 'LowPrice', 'ClosePrice', 'TotalVolume', 
                                     'TotalQuantity', 'TotalTradeCount'])
    key = 0
    ind = 0
    while ind <= len(df):
        if ind == len(indexs):
            ndf = df[key:]
            work_df = ndf.copy()
            df_final = df_final.append(work_df,ignore_index=True)
            break
        #    ndf = df[key:]
        
        ####Agregarmos columnas####
        ndf = df[key:indexs[ind]+1]
        work_df = ndf.copy()
        num = 0
        while num < num_rows[ind]:
            row = df[len(ndf)-1:len(ndf)]
            
            work_df.loc[len(work_df)] = [row['Date'][len(work_df)-1],
                                         (parse_time( row['Timestamp'][len(work_df)-1]  ) + timedelta(minutes=num+1)).strftime('%H:%M:%S'),
                                         row['Ticker'][len(work_df)-1],
                                         row['OpenPrice'][len(work_df)-1], 
                                         row['HighPrice'][len(work_df)-1], 
                                         row['LowPrice'][len(work_df)-1], 
                                         row['ClosePrice'][len(work_df)-1], 
                                         0, 0, 0]
            num = num + 1
        key = indexs[ind] + 1
        ind = ind +1
        df_final = df_final.append(work_df,ignore_index=True)
        ###########################
    #print(df_final)    
    df_final.to_csv('./emisoras/oc.csv', encoding='utf-8', index=False)
             

In [17]:
file = pd.read_csv('./emisoras/All-Clean-Ticker-Data/AME.csv')
df = pd.DataFrame(file)
index = get_index_optimizado(df)
index
#optimizacion(df,index)

[{'index': 39, 'num': 1.0},
 {'index': 141, 'num': 1.0},
 {'index': 159, 'num': 1.0},
 {'index': 164, 'num': 1.0},
 {'index': 170, 'num': 1.0},
 {'index': 176, 'num': 1.0},
 {'index': 178, 'num': 1.0},
 {'index': 183, 'num': 2.0},
 {'index': 184, 'num': 1.0},
 {'index': 187, 'num': 1.0},
 {'index': 191, 'num': 1.0},
 {'index': 193, 'num': 1.0},
 {'index': 200, 'num': 1.0},
 {'index': 202, 'num': 1.0},
 {'index': 205, 'num': 1.0},
 {'index': 206, 'num': 2.0},
 {'index': 222, 'num': 1.0},
 {'index': 241, 'num': 2.0},
 {'index': 245, 'num': 1.0},
 {'index': 253, 'num': 1.0},
 {'index': 276, 'num': 1.0},
 {'index': 279, 'num': 1.0},
 {'index': 308, 'num': 1.0},
 {'index': 331, 'num': 1.0},
 {'index': 372, 'num': 1.0},
 {'index': 385, 'num': 1.0},
 {'index': 398, 'num': 1.0},
 {'index': 427, 'num': 1.0},
 {'index': 487, 'num': 1.0},
 {'index': 552, 'num': 1.0},
 {'index': 555, 'num': 1.0},
 {'index': 559, 'num': 1.0},
 {'index': 571, 'num': 1.0},
 {'index': 574, 'num': 1.0},
 {'index': 773,

In [19]:
#optimizacion(df,index)

In [20]:
for ticker in ls('./emisoras/All-Complete-Ticker-Data/'):
    file = pd.read_csv('./emisoras/All-Complete-Ticker-Data/' + ticker)
    df = pd.DataFrame(file)
    index = get_index_optimizado(df) 
    print(index)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
